## **Librerias**

In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
import sklearn.decomposition
import matplotlib.pyplot as plt
import math
import statistics
import sklearn
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import VarianceThreshold ##new added##
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, completeness_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import tree
import graphviz
import pydot
import pydotplus
from sklearn import svm
import os
import seaborn as sns

## **Carga de datos**

In [ ]:
fraud_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fraud_data_complete_final.csv")
fraud_data = fraud_data.drop(fraud_data.columns[0], axis=1)
fraud_data.head(7)

,ID_USER,genero,monto,fecha,hora,dispositivo,establecimiento,ciudad,tipo_tc,linea_tc,interes_tc,status_txn,is_prime,dcto,cashback,fraude
0,0,F,608.345633,2020-01-21,20,ANDROID,Super,Merida,Fisica,71000.0,51.0,Aceptada,False,60.834563,5.475111,False
1,0,F,88.719243,2020-01-15,7,ANDROID,MPago,Merida,Virtual,71000.0,51.0,Aceptada,False,0.000000,1.774385,False
2,1,F,790.037653,2020-01-17,11,ANDROID,MPago,Guadalajara,Virtual,94000.0,56.0,En proceso,False,0.000000,15.800753,False
3,1,F,767.005387,2020-01-05,11,WEB,MPago,Guadalajara,Fisica,94000.0,56.0,Aceptada,False,0.000000,7.670054,False
4,1,F,254.171053,2020-01-30,10,ANDROID,Farmacia,Guadalajara,Fisica,94000.0,56.0,Aceptada,False,0.000000,2.541711,False
5,1,F,37.003747,2020-01-27,3,ANDROID,MPago,Guadalajara,Fisica,94000.0,56.0,Rechazada,False,0.000000,0.370037,False
6,1,F,334.516017,2020-01-06,2,ANDROID,Abarrotes,Guadalajara,Fisica,94000.0,56.0,Aceptada,False,33.451602,3.010644,False


In [ ]:
fraud_data.shape

(6909, 16)

In [ ]:
fraud_data.dtypes

ID_USER              int64
genero              object
monto              float64
fecha               object
hora                 int64
dispositivo         object
establecimiento     object
ciudad              object
tipo_tc             object
linea_tc           float64
interes_tc         float64
status_txn          object
is_prime              bool
dcto               float64
cashback           float64
fraude                bool
dtype: object

## **Establecer variables predictoras y variable objetivo**

In [ ]:
y = fraud_data.fraude

fraud_data_features = ['dispositivo', 'establecimiento','ciudad', 'tipo_tc', 'linea_tc','interes_tc', 'is_prime']

X = fraud_data[fraud_data_features]

In [ ]:
X

,dispositivo,establecimiento,ciudad,tipo_tc,linea_tc,interes_tc,is_prime
0,ANDROID,Super,Merida,Fisica,71000.0,51.0,False
1,ANDROID,MPago,Merida,Virtual,71000.0,51.0,False
2,ANDROID,MPago,Guadalajara,Virtual,94000.0,56.0,False
3,WEB,MPago,Guadalajara,Fisica,94000.0,56.0,False
4,ANDROID,Farmacia,Guadalajara,Fisica,94000.0,56.0,False
...,...,...,...,...,...,...,...
6904,WEB,Farmacia,Merida,Virtual,94000.0,45.0,False
6905,ANDROID,Farmacia,Merida,Virtual,94000.0,45.0,False
6906,WEB,Super,Merida,Fisica,94000.0,45.0,False
6907,ANDROID,MPago,Monterrey,Fisica,94000.0,45.0,False


## **Dividir datos para entrenamiento y validacion**

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

## **One Hot Encoding**

Establecer variables numericas y categoricas

In [ ]:
numeric_cols = train_X.select_dtypes(include=['float64', 'int', 'bool']).columns.to_list()
cat_cols = train_X.select_dtypes(include=['object', 'category']).columns.to_list()

Usar OneHotEncoder en las variables categoricas



In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
preprocessor = ColumnTransformer(
                   [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                remainder='passthrough')

train_X_prep = preprocessor.fit_transform(train_X)
val_X_prep  = preprocessor.transform(val_X)

In [ ]:
train_X_prep

array([[0.0, 1.0, 1.0, ..., 48000.0, 57.0, False],
       [1.0, 0.0, 0.0, ..., 42000.0, 56.0, True],
       [1.0, 0.0, 0.0, ..., 80000.0, 52.0, False],
       ...,
       [0.0, 1.0, 0.0, ..., 45000.0, 63.0, False],
       [1.0, 0.0, 0.0, ..., 82000.0, 47.0, False],
       [0.0, 1.0, 0.0, ..., 26000.0, 51.0, False]], dtype=object)

###**Analisis de varianza en las variables predictoras**

- Se busca describir las varianza como altas y bajas, donde True : Baja varianza y False: Alta varianza

In [ ]:
selector = VarianceThreshold()
selector.fit(train_X_prep)
selector.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

- Colocamos en formato de dataframe con los nombres de la columnas, para visualizar mejor cuales son las variables con alta y baja varianza.

In [ ]:
encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names(cat_cols)
labels = np.concatenate([encoded_cat, numeric_cols])

datos_val_prep = preprocessor.transform(val_X)
datos_val_prep = pd.DataFrame(val_X_prep, columns=labels)

datos_train_prep = preprocessor.transform(train_X)
datos_train_prep = pd.DataFrame(train_X_prep, columns=labels)
datos_train_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5181 entries, 0 to 5180
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   dispositivo_ANDROID          5181 non-null   object
 1   dispositivo_WEB              5181 non-null   object
 2   establecimiento_Abarrotes    5181 non-null   object
 3   establecimiento_Farmacia     5181 non-null   object
 4   establecimiento_MPago        5181 non-null   object
 5   establecimiento_Restaurante  5181 non-null   object
 6   establecimiento_Super        5181 non-null   object
 7   ciudad_Guadalajara           5181 non-null   object
 8   ciudad_Merida                5181 non-null   object
 9   ciudad_Monterrey             5181 non-null   object
 10  ciudad_Toluca                5181 non-null   object
 11  tipo_tc_Fisica               5181 non-null   object
 12  tipo_tc_Virtual              5181 non-null   object
 13  linea_tc                     5181

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


- Imprimimos las variables con alta varianza

In [ ]:
selector = VarianceThreshold()
selector.fit(datos_train_prep)
concol = [column for column in datos_train_prep.columns 
          if column not in datos_train_prep.columns[selector.get_support()]]

print(concol)

[]


Como se puede verificar, ninguna de las variables tiene alta varianza, lo cual podria afectar al modelo. Es importante que se verifique este procedimiento antes de hacer la estandarizacion.

## **Estandarización de las variables numericas**

Usar StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
# Se estandarizan las columnas numéricas

preprocessor = ColumnTransformer(
                   [('scale', StandardScaler(), numeric_cols)],
                remainder='passthrough')

In [ ]:
train_X_prep = preprocessor.fit_transform(datos_train_prep)
val_X_prep  = preprocessor.transform(datos_val_prep)

In [ ]:
train_X_prep

array([[-0.6470078656153512, 0.9454937610043455, -0.3880287220562067,
        ..., 0.0, 0.0, 1.0],
       [-0.9167767173579179, 0.8411638315906017, 2.577128813302506, ...,
        0.0, 0.0, 1.0],
       [0.791759343678338, 0.4238441139356265, -0.3880287220562067, ...,
        0.0, 1.0, 0.0],
       ...,
       [-0.7818922914866345, 1.5714733374868082, -0.3880287220562067,
        ..., 0.0, 0.0, 1.0],
       [0.8816822942591936, -0.09780553313309243, -0.3880287220562067,
        ..., 0.0, 1.0, 0.0],
       [-1.6361603220047625, 0.31951418452188274, -0.3880287220562067,
        ..., 1.0, 1.0, 0.0]], dtype=object)

In [ ]:
val_X_prep

array([[0.7018363930974825, 0.8411638315906017, -0.3880287220562067, ...,
        0.0, 0.0, 1.0],
       [-1.1415840938100568, -0.6194551802018113, -0.3880287220562067,
        ..., 0.0, 1.0, 0.0],
       [1.0165667201304769, -0.9324449684430427, -0.3880287220562067,
        ..., 0.0, 1.0, 0.0],
       ...,
       [-0.7818922914866345, 1.2584835492455768, 2.577128813302506, ...,
        0.0, 0.0, 1.0],
       [1.421219997744327, -0.5151252507880676, -0.3880287220562067, ...,
        0.0, 1.0, 0.0],
       [0.9716052448400492, 0.21518425510813893, -0.3880287220562067,
        ..., 0.0, 0.0, 1.0]], dtype=object)

# **Evaluando la prediccion con diferentes modelos**

## **Random Forest Model**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#max_depth  = 100, n_estimators =1000, n_jobs=1, min_samples_leaf=200
randomforest_model = RandomForestRegressor(n_estimators=50, random_state=0)
randomforest_model.fit(train_X_prep, train_y)
preds = randomforest_model.predict(val_X_prep)

print("Mean Absolute Error:") 
print(mean_absolute_error(val_y, preds))

Mean Absolute Error:
0.074184596043103


In [ ]:
preds = pd.DataFrame(preds)
preds = round(preds, 0)
preds = preds.replace(0, False)
preds = preds.replace(1, True)
preds

In [ ]:
confMat_RFM = sklearn.metrics.confusion_matrix(val_y, preds)
confMatList_RFM = confMat_RFM.tolist()
TN = confMatList_RFM[0][0]
TP = confMatList_RFM[1][1]
FN = confMatList_RFM[1][0]
FP = confMatList_RFM[0][1]

accuracy_RFM = (TP + TN)/(TP + FP + FN + TN)
recall_RFM = (TP) / (TP + FN)
print("Accuracy:", accuracy_RFM)
print("-"*50)
if (TP + FP) == 0:
  print("Durante la prediccion no detectó fraudes a pesar de su existencia. No es posible calcular la precision")
else:
  precision_RFM = (TP)/(TP + FP)
  print("Precision: ", precision_RFM)
print("-"*50)
print("Recall:", recall_RFM)
print("-"*50)
confMat_RFM

Accuracy: 0.9594907407407407
--------------------------------------------------
Precision:  0.0
--------------------------------------------------
Recall: 0.0
--------------------------------------------------


array([[1658,   12],
       [  58,    0]])

## **Neural Network**

In [ ]:
neuralnetwork_model = MLPRegressor()
neuralnetwork_model.fit(train_X_prep, train_y)
y_pred_nn = neuralnetwork_model.predict(val_X_prep)
y_pred_nn = np.rint(y_pred_nn)

confMat_nn = sklearn.metrics.confusion_matrix(val_y, y_pred_nn)
confMatList_nn = confMat_nn.tolist()
TN = confMatList_nn[0][0]
TP = confMatList_nn[1][1]
FN = confMatList_nn[1][0]
FP = confMatList_nn[0][1]

accuracy_nn = (TP + TN)/(TP + FP + FN + TN)
recall_nn = (TP) / (TP + FN)
print("Accuracy:", accuracy_nn)
print("-"*50)
if (TP + FP) == 0:
  print("Durante la prediccion no detectó fraudes a pesar de su existencia. No es posible calcular la precision")
else:
  precision_nn = (TP)/(TP + FP)
  print("Precision: ", precision_nn)
print("-"*50)
print("Recall:", recall_nn)
print("-"*50)
confMat_nn

Accuracy: 0.9664351851851852
--------------------------------------------------
Durante la prediccion no detectó fraudes a pesar de su existencia. No es posible calcular la precision
--------------------------------------------------
Recall: 0.0
--------------------------------------------------


array([[1670,    0],
       [  58,    0]])

## **Regresion logistica**

In [ ]:
reglog_model = sklearn.linear_model.LogisticRegression()
reglog_model.fit(train_X_prep, train_y)

y_pred_reglog = reglog_model.predict(val_X_prep)
confmat_reglog = sklearn.metrics.confusion_matrix(val_y, y_pred_reglog)
confmatlist_reglog = confmat_reglog.tolist()
TN = confmatlist_reglog[0][0]
TP = confmatlist_reglog[1][1]
FN = confmatlist_reglog[1][0]
FP = confmatlist_reglog[0][1]

accuracy_reglog = (TP + TN)/(TP + FP + FN + TN)
recall_reglog = (TP) / (TP + FN)

print("Accuracy:", accuracy_reglog)
print("-"*50)
if (TP + FP) == 0:
  print("Durante la prediccion no detectó fraudes a pesar de su existencia. No es posible calcular la precision")
else:
  precision_reglog = (TP)/(TP + FP)
  print("Precision: ", precision_reglog)
print("-"*50)
print("Recall:", recall_reglog)
print("-"*50)
confmat_reglog

Accuracy: 0.9664351851851852
--------------------------------------------------
Durante la prediccion no detectó fraudes a pesar de su existencia. No es posible calcular la precision
--------------------------------------------------
Recall: 0.0
--------------------------------------------------


array([[1670,    0],
       [  58,    0]])

# **¿Cuán bueno es el modelo?**

De los 3 modelos, el modelo predictivo usando Random Forest arroja datos interpretables para el 'precision' a diferencia de los demas modelos que arrojan datos no deterministicos. Sin embargo, en la deteccion de fraudes verdaderos Random Forest no detecta algún caso a pesar de su existencia. Esto podría deberse a la varianza presente en la variables predictoras, puesto que se encontró que todas tenian varianza baja.

In [ ]:
comparar_tabla = [['Accuracy', round(accuracy_RFM, 2),round(accuracy_nn, 2),round(accuracy_reglog, 2)], ['Precision', precision_RFM,'no determinado','no determinado'], ['Recall',recall_RFM,recall_nn,recall_reglog]]
comparar_tabla = pd.DataFrame(comparar_tabla)
comparar_tabla.columns = ['Metrica', 'Random Forest', 'Neural Network', 'Regresion Logistica']
comparar_tabla

,Metrica,Random Forest,Neural Network,Regresion Logistica
0,Accuracy,0.96,0.97,0.97
1,Precision,0.00,no determinado,no determinado
2,Recall,0.00,0,0


# **¿Cuáles son las características más importantes y cómo los valores de las variables influyen en el modelo?**

Acorde al análisis exploratorio de lo datos, se encontró que las variables 'dispositivo', 'establecimiento','ciudad', 'tipo_tc', 'linea_tc','interes_tc' y 'is_prime' presentaban distribuciones interesantes en los casos de fraude (3.3% del total), aunque no se evidenciaron correlacion fuertes en el mapa de calor. La inclusion de estas variables en el modelo para predecir la presencia de fraude no ha evidenciado buenos resultados en los diferentes modelos evaluados (Random Forest, Neural Network y Regresion Logistica). Ello puede deberse a las caracteristicas de las variables, ya que sus varianzas son bajas. Se recomendaria continuar el analisis, donde pueda resolverse el tema de las varianzas y reevaluar el rendimiento.

# **Explique la compensación de usar su modelo para detectar estafadores** 